# Twitter Video Sentiment Analysis Tool
This tool performs sentiment analysis on Twitter videos to determine the polarity of users' responses to given topics. More specifically, the program detects & analyzes facial expressions in videos and assigns scores according to several classes of emotions.  

**Parameters:**  
- `search_terms`: A list of terms to search through, gather videos from, and report scores on.  

**Output:**  
- `results`: A list of dataframes containing the average distribution of emotions for each search term. 

In [7]:
import snscrape.modules.twitter as snt
import urllib.request
import pandas as pd
import numpy as np
from fer import FER
from fer import Video
import os

# Twitter search terms can be hashtags or words/phrases
search_terms = ['#COVIDBooster', 'COVID booster']

In [8]:
# Returns a list of video links given a search term
def get_videos(search_term):
    videos = []
    vid_count = 0
    for tweet in snt.TwitterSearchScraper(search_term).get_items():
        ### FOR DEMONSTRATION PURPOSES, WE WILL LIMIT THE INPUT LIST TO 5 VIDEOS PER SEARCH TERM ###
        if vid_count >= 5: return videos
        if tweet.media:
            for medium in tweet.media:
                if isinstance(medium, snt.Video) and ".mp4" in medium.variants[0].url and medium.variants[0].url not in videos:
                    videos.append(medium.variants[0].url)
                    vid_count += 1
    return videos

In [9]:
# Runs facial emotion recognition on a video and returns emotion scores
def get_scores(video_path):
    face_detector = FER()
    input_video = Video(video_path)

    # Analyze video, store results in dataframe
    processing_data = input_video.analyze(face_detector, save_frames=False, save_video=False, annotate_frames=False)
    vid_df = input_video.to_pandas(processing_data)
    vid_df = input_video.get_first_face(vid_df)
    vid_df = input_video.get_emotions(vid_df)

    # Categorize emotion scores
    angry = sum(vid_df.angry)
    disgust = sum(vid_df.disgust)
    fear = sum(vid_df.fear)
    happy = sum(vid_df.happy)
    sad = sum(vid_df.sad)
    surprise = sum(vid_df.surprise)
    neutral = sum(vid_df.neutral)
    emotion_values = np.array([angry, disgust, fear, happy, sad, surprise, neutral])
    
    return emotion_values

In [10]:
# Finds the average emotion scores for a search term
def avg_scores(search_term):
    video_path = "input_video.mp4"
    print(f'Getting videos for "{search_term}"...')
    videos = get_videos(search_term)
    scores = np.zeros(7)
    
    for i, url in enumerate(videos):
        try:
            # Download video for analysis purposes
            urllib.request.urlretrieve(url, video_path)
            
            # Update average scores
            print(f"Calculating scores for video {i+1}/{len(videos)}")
            scores = np.average([scores, get_scores(video_path)], axis=0)
        except Exception as e:
            print(e)
        finally:
            # Cleanup
            if os.path.exists(video_path):
                os.remove(video_path) 
            if os.path.exists("data.csv"):
                os.remove("data.csv")
            if os.path.exists("output"):
                os.rmdir("output")
            continue
            
    return scores

In [11]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
results = []

# Display the average scores per search term as percentages
for term in search_terms:
    # Determine order in which to display emotions
    scores = avg_scores(term)
    emotion_order = [emotion for value, emotion in sorted(zip(scores, emotions), reverse=True)]
    
    # Convert scores to percentages
    total = sum(scores)
    scores = sorted([value / total * 100 for value in scores], reverse=True)
    
    # Display results in a dataframe
    result = pd.DataFrame(emotion_order, columns = ['Emotion'])
    result[f"Prevalence in {term}"] = [f'{num:.3g}' + '%' for num in scores]
    results.append(result)

Getting videos for "#COVIDBooster"...
Calculating scores for video 1/5
29-11-2021:02:16:38,324 INFO     [classes.py:199] 25.00 fps, 250 frames, 10.00 seconds


100%|██████████| 250/250 [00:20<00:00, 12.17frames/s]


Calculating scores for video 2/5
29-11-2021:02:17:00,9 INFO     [classes.py:199] 29.97 fps, 1364 frames, 45.51 seconds


100%|██████████| 1364/1364 [00:27<00:00, 49.41frames/s]


Calculating scores for video 3/5
29-11-2021:02:17:28,478 INFO     [classes.py:199] 30.00 fps, 286 frames, 9.53 seconds


...

100%|██████████| 3308/3308 [06:39<00:00,  8.28frames/s]


Calculating scores for video 4/5
29-11-2021:02:34:15,382 INFO     [classes.py:199] 29.97 fps, 1348 frames, 44.98 seconds


100%|██████████| 1348/1348 [00:58<00:00, 23.08frames/s]


Calculating scores for video 5/5
29-11-2021:02:35:15,73 INFO     [classes.py:199] 34.39 fps, 4213 frames, 122.49 seconds


100%|██████████| 4213/4213 [10:03<00:00,  6.98frames/s]


In [12]:
# Display the resulting score percentages for each search term
for result in results:
    display(result)

,Emotion,Prevalence in #COVIDBooster
0,Fear,33.6%
1,Sad,22.3%
2,Angry,19%
3,Neutral,16%
4,Happy,6.7%
5,Surprise,2%
6,Disgust,0.336%


,Emotion,Prevalence in COVID booster
0,Angry,26.6%
1,Neutral,21.9%
2,Fear,17%
3,Sad,16.6%
4,Happy,13.4%
5,Surprise,3.44%
6,Disgust,1.06%
